In [1]:
# signateのデータに対してCLDNNsを実装

In [2]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display # 波形のプロットに必要
import IPython.display as ipd #jupyter-notebook上で音声再生
import glob
import re
import seaborn as sns
import pickle
import scipy.signal as ss
import os

# keras
import keras
#from keras.datasets import mnist
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout,Input,Flatten,Activation
from keras.layers import Conv1D,Conv2D,MaxPooling1D,BatchNormalization,Reshape
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam, SGD
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Model

# モデル可視化用
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from tensorflow.python import debug as tf_debug

Using TensorFlow backend.


In [3]:
# jupyter-notebookのcellの幅を広げる
from IPython.core.display import display,HTML
display(HTML("<style>.container{width:100%!important;}</style>"))

In [4]:
# 乱数のシードを固定
np.random.seed(123)

In [5]:
# 作業ディレクトリの設定
os.chdir("/home/taichi/DataAnalysis/05_NTT_corevo")

In [6]:
train_info = pd.read_csv("01_input/ntt_corevo/class_train.tsv",
                        delimiter = "\t",
                        names = ["filename","label"])

train_info["raw_wave_path"] = "03_work/raw_waves/train/" + train_info["filename"] + ".pickle"

In [7]:
train_info.head()

,filename,label,raw_wave_path
0,0002f1cd968ca78ada9e1c7037224773,MA_CH,03_work/raw_waves/train/0002f1cd968ca78ada9e1c...
1,0003747ec9268461d4cbb9e1b86e9663,FE_AD,03_work/raw_waves/train/0003747ec9268461d4cbb9...
2,0003b32f378b001f0f73bf0981da8773,MA_CH,03_work/raw_waves/train/0003b32f378b001f0f73bf...
3,0004ab975bf8b59e1b19f2b7b6d1548b,MA_CH,03_work/raw_waves/train/0004ab975bf8b59e1b19f2...
4,0005678b57ca265a65f8ef0cc7481277,MA_AD,03_work/raw_waves/train/0005678b57ca265a65f8ef...


In [8]:
train_info["raw_wave_path"][0]

'03_work/raw_waves/train/0002f1cd968ca78ada9e1c7037224773.pickle'

In [9]:
# ファイルが存在するか確認
#!ls -lad 03_work/raw_waves/train/0002f1cd968ca78ada9e1c7037224773.pickle 

In [10]:
train_info.head()

,filename,label,raw_wave_path
0,0002f1cd968ca78ada9e1c7037224773,MA_CH,03_work/raw_waves/train/0002f1cd968ca78ada9e1c...
1,0003747ec9268461d4cbb9e1b86e9663,FE_AD,03_work/raw_waves/train/0003747ec9268461d4cbb9...
2,0003b32f378b001f0f73bf0981da8773,MA_CH,03_work/raw_waves/train/0003b32f378b001f0f73bf...
3,0004ab975bf8b59e1b19f2b7b6d1548b,MA_CH,03_work/raw_waves/train/0004ab975bf8b59e1b19f2...
4,0005678b57ca265a65f8ef0cc7481277,MA_AD,03_work/raw_waves/train/0005678b57ca265a65f8ef...


In [11]:
# label2dictをよみこみ
with open("03_work/label2int.pickle",mode = "rb") as f:
    label2int = pickle.load(f)

In [12]:
label2int

{'FE_AD': 0, 'FE_CH': 1, 'FE_EL': 2, 'MA_AD': 3, 'MA_CH': 4, 'MA_EL': 5}

In [13]:
train_info.head()

,filename,label,raw_wave_path
0,0002f1cd968ca78ada9e1c7037224773,MA_CH,03_work/raw_waves/train/0002f1cd968ca78ada9e1c...
1,0003747ec9268461d4cbb9e1b86e9663,FE_AD,03_work/raw_waves/train/0003747ec9268461d4cbb9...
2,0003b32f378b001f0f73bf0981da8773,MA_CH,03_work/raw_waves/train/0003b32f378b001f0f73bf...
3,0004ab975bf8b59e1b19f2b7b6d1548b,MA_CH,03_work/raw_waves/train/0004ab975bf8b59e1b19f2...
4,0005678b57ca265a65f8ef0cc7481277,MA_AD,03_work/raw_waves/train/0005678b57ca265a65f8ef...


In [14]:
# batchsizeで割り切れない場合を考慮しないといけない。

In [15]:
# filepathとlabelを入力として受け取るRAW_WAVE_GENERATORを実装
class RawWaveGenerator(keras.callbacks.Callback):
    def __init__(self,minibatchsize,duration_sec,data_df):
        self.duration_sec = duration_sec
        self.minibatchsize = minibatchsize 
        self.cur_index = 0
      #  self.wave_length = wave_length
#        self.step = step
        self.data_df = data_df
        self.data_size = data_df.shape[0]
        self.iteration = np.floor(self.data_size/minibatchsize)
        self.sr = 22050
    def shuffle_df(self):
        # 学習データの学習順序をシャッフル
        self.data_df = self.data_df.sample(frac=1).reset_index(drop=True) #.loc[self.cur_index:self.cur_index+self.minibatchsize-1,:]
    
    def get_batch(self,minibatchsize,duration_sec):
        inputs = np.zeros([minibatchsize,duration_sec*self.sr,1])
        outputs = np.zeros(minibatchsize)
        
        train_batch_df = self.data_df.loc[self.cur_index:self.cur_index+self.minibatchsize-1,:]
#        print(self.cur_index)
#        print(self.cur_index+minibatchsize-1)
#       print(train_batch_df)
        for i,(index,v) in enumerate(train_batch_df.iterrows()):
            #    print(v)
           # print(i)
            label = v["label"]
            wave_path = v["raw_wave_path"]
    
            with open(wave_path,mode="rb") as f:
                tmp = pickle.load(f)
                sr = tmp["sampling_rate"]
                raw_wave = tmp["raw_wave"]

            # 基本的にサンプリングレートは22050のはず
            assert sr == 22050

            # 変数名aは小さなスコープしかないから許して
            a = raw_wave[:duration_sec*sr]
            # データがduration秒
            a = np.pad(a, [0,duration_sec*sr-len(a)],'constant')
            
#            print(a.shape)
#            print(inputs.shape)
#            print(i)

            inputs[i,:,0] = a
            outputs[i] = label2int[label]
            
        self.cur_index = self.cur_index + minibatchsize
        
        return (inputs,outputs)
                
    def next_train(self):
        self.cur_index = 0
        self.shuffle_df()
        
        while True:    
            if self.cur_index / self.minibatchsize > self.iteration:
                self.cur_index = 0
            yield self.get_batch(minibatchsize = self.minibatchsize,
                                 duration_sec = self.duration_sec)

#    def next_val(self):
#        while True:           
#            yield self.get_batch(minibatchsize = self.minibatchsize,
#                                 duration_sec = self.duration_sec)


In [16]:
data_gen = RawWaveGenerator(minibatchsize=32,duration_sec=5,data_df=train_info)

In [17]:
# np.max([np.nan,np.inf])

In [18]:
#for i,v in enumerate(data_gen.next_train()):
    # 音声データの振幅がinfやnanでないか確認
#    print(np.min(v[0]))
#    if i == 5:
#        break

In [19]:
data_gen.cur_index

0

In [20]:
# tensorflowのデバッガを設定。lossがnanやinfになるとき、役に立つ
def set_debugger_session():
    sess = K.get_session()
    sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.add_tensor_filter('has_inf_or_nan', tf_debug.has_inf_or_nan)
    K.set_session(sess)

In [21]:
# 下記デバッガはjupyter-notebook上で実行すると、ターミナル上で実行される。意外と便利
#set_debugger_session()

In [22]:
# CLDNNsモデルの構築

In [23]:
# https://stackoverflow.com/questions/43915482/how-do-you-create-a-custom-activation-function-with-keras
# 上記参考
def log_relu(x):
    return(K.log(K.relu(x)+0.001))

In [24]:
# 自動でreshapeする関数を作成

In [38]:

# ラベルの数
output_dim = len(label2int)

#学習モデルの構築
sr = 22050
duration_sec = 1
#words_per_epoch = 16000
val_split = 0.2
#val_words = int(words_per_epoch * (val_split))

#conv_filters = 256
conv_filters = 400
#kernel_size = (3, 3)
kernel_size = 3
pool_size = 2
time_dense_size = 32

window_size_sec = 35 * (10 ** -3)

#rnn_size = 
#minibatchsize = 32

input_shape = [duration_sec*sr,1]
    
# TODO : Conv1Dに変更    

act = 'relu'

input_data = Input(name='the_input', 
                   shape=input_shape, 
                   dtype='float32')

inner = Conv1D(filters = conv_filters, 
               kernel_size = int(window_size_sec * sr),
               padding='valid',
               kernel_initializer='glorot_uniform',
               name='conv1')(input_data)

inner = MaxPooling1D(pool_size= duration_sec*sr - int(window_size_sec * sr) + 1, name='max1')(inner)

inner = Activation(log_relu,name="act1")(inner)
#inner = Activation("relu",name="act1")(inner)
#inner = Activation(log_relu,name="relu")(inner)

inner = Reshape((conv_filters,1),name="reshape1")(inner)

inner = Conv1D(filters=256,
               kernel_size= 8, padding='valid',
               kernel_initializer='glorot_uniform',
               activation="relu",
               name='conv2')(inner)

#inner = Activation(log_relu,name="act2")(inner)

inner = MaxPooling1D(pool_size=3, 
                     name='max2')(inner)

timesteps = 32
data_dim = 512
rnn_size = 832

inner = LSTM(rnn_size, return_sequences=True,
             input_shape=(timesteps, data_dim),
            name = "lstm1")(inner)

inner = LSTM(rnn_size, return_sequences=True,
             input_shape=(timesteps, data_dim),
            name = "lstm2")(inner)

inner = LSTM(rnn_size, return_sequences=True,
             input_shape=(timesteps, data_dim),
            name = "lstm3")(inner)

inner = Flatten(name = "flatten")(inner)

inner = Dense(1024,name = "dense1")(inner)

inner = BatchNormalization(name="batch_norm1")(inner)

inner = Activation("relu",name="act2")(inner)

out = Dense(output_dim,activation="sigmoid",name = "dense2")(inner)

model = Model(inputs=input_data, outputs=out)

sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

model.compile(loss = "sparse_categorical_crossentropy", optimizer=sgd,metrics = ["accuracy"])


In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 22050, 1)          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 21280, 400)        308800    
_________________________________________________________________
max1 (MaxPooling1D)          (None, 1, 400)            0         
_________________________________________________________________
act1 (Activation)            (None, 1, 400)            0         
_________________________________________________________________
reshape1 (Reshape)           (None, 400, 1)            0         
_________________________________________________________________
conv2 (Conv1D)               (None, 393, 256)          2304      
_________________________________________________________________
max2 (MaxPooling1D)          (None, 131, 256)          0         
__________

In [40]:
train_info_shuffle = train_info.sample(frac=1.0).reset_index(drop = True)

In [41]:
train_rate = 0.8

In [42]:
train_df = train_info_shuffle.loc[0:int(train_info.shape[0]*train_rate),]
val_df = train_info_shuffle.loc[int(train_info.shape[0]*train_rate) + 1:,]

In [43]:
minibatchsize = 5
train_gen = RawWaveGenerator(minibatchsize=minibatchsize,duration_sec=duration_sec,data_df=train_df)
val_gen = RawWaveGenerator(minibatchsize=minibatchsize,duration_sec=duration_sec,data_df=val_df)

In [44]:
start_epoch = 0
epochs = 100
hist = model.fit_generator(generator=train_gen.next_train(),
                               steps_per_epoch=1000,
                               epochs=100,
                               validation_data=val_gen.next_train(),
                               validation_steps=50,
                               initial_epoch=start_epoch)

Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[108992,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/SGD/Square_13}} = Square[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/SGD/gradients/dense1_3/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
